In [ ]:
!pip install ipython-autotime
%load_ext autotime

# Train a GPT-2 Text-Generating Model 

<a href="https://github.com/DrSnowbird/gpt-2-simple-docker/notebooks/GPT-2-Text-Generating-Model-with-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to run the [GPT-2 Text-Generating Model demo](https://github.com/DrSnowbird/gpt-2-simple-docker/). See the link for more details about the model, including evaluation metrics and credits.

# Modifications by @DrSnowbird
1. This notebook has been minorly modifed by @DrSnowbird to use **Tensorflow v2.x**
2. Steps is reduced from **1,000 down to 200** since this Demo is just to show concept not for achieving good accuracy fine-tuned model.
3. Removed the use of Google-Drive to simplify and avoid the problem in the gpt-2-simple copy-from-gdrive with hard-code path 'My Drive' which not existing.

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 2.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Thu Dec  9 01:57:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`, e.g., `/models/124M`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 261Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 8.21Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 808Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:12, 39.8Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 650Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.86Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 9.97Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
# gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O "shakespeare.txt"

--2021-12-09 01:57:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2021-12-09 01:57:23 (21.1 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [ ]:
file_name = "shakespeare.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
#gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=200,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=100,
              save_every=200
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


dataset has 338025 tokens
Training...
[10 | 60.28] loss=3.46 avg=3.46
[20 | 115.55] loss=3.59 avg=3.53
[30 | 170.91] loss=3.15 avg=3.40
[40 | 226.18] loss=3.48 avg=3.42
[50 | 281.36] loss=3.14 avg=3.36
[60 | 336.56] loss=3.29 avg=3.35
[70 | 391.79] loss=3.25 avg=3.34
[80 | 446.97] loss=3.31 avg=3.33
[90 | 502.19] loss=2.96 avg=3.29
[100 | 557.45] loss=3.05 avg=3.27
======== SAMPLE 1 ========
 I know I am in debt of it, and cannot bear it, and that I fear may prove to be grievous, or else, in that the issue of the money I issue from the people, I shall be found in debt.

WARWICK:
Now, sir, you shall pay, as you will; and all the rest shall be paid.

Nurse:
Sir, I have a son, Henry, named Richard, and shall be my wife.

DUKE VINCENTIO:
I thank you, sir.

WARWICK:
It was well said by me, I have been born a Christian, and taught you for many years in the diocese,
but to no great height.

DUKE VINCENTIO:
I hope I shall never be more.

Nurse:
Sir, how is it, sir: here is a letter I have been

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
#gpt2.copy_checkpoint_to_gdrive(run_name='run1')


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
# This line is commented by our DrSnowbird!
#
# gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (**Runtime -> Restart Runtime**). Don't 'terminate' colab session - or, your will lost the fine-tuned trained model above!

**Otherwise, the 'load_gpt2(sess, ...)' will fail**!

**You will need to rerun imports as the following parts of code.**

In [ ]:
%tensorflow_version 2.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-300
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-300


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

With the new-made law,
The divorcee is the widow;
And the court's gaoler is the woman
As the bride's court.
What, in a bridal bed!

KING HENRY VI:
I do beseech you, think not I am gone.

CLIFFORD:
You say that I am gone.

KING HENRY VI:
O, think not you.

CLIFFORD:
I am gone.

KING HENRY VI:
I take my leave now.

CLIFFORD:
I am gone.

KING HENRY VI:
Why, then I am gone.

Clown:
O that only the couple should depart,
And be gone.

CLIFFORD:
I am gone.

KING HENRY VI:
And you the former.

CLIFFORD:
I am gone.

KING HENRY VI:
But you the former.

CLIFFORD:
I am gone.

KING HENRY VI:
And you the former.

Clown:
O, that was a very pretty play.

KING HENRY VI:
Ay, but a pretty one.

Clown:
We have not been to dinner.

KING HENRY VI:
Why, why, why, why, why, why, why, why, why, why, why, why?

Clown:
Why, why, why, why, why, why, why?

KING HENRY VI:
Why, why, why, why, why, why?

Clown:
Why, why, why, why, why, why?

KING HENRY VI:
Why, why, why, why, why, why?

Clown:
Why, why, why, why, why

In [ ]:
def saveListToFile(textList, outFileName):
    with open(outFileName, 'w') as f:
        f.write(textList)

In [ ]:
text = gpt2.generate(sess, return_as_list=True)[0]
saveListToFile(text, 'generated-text-01.txt')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5)

LORD FITZWATER:
A stone is thrown to his head,
That blasts forth a loud cry;
That is my voice, if this be true.

LORD FITZWATER:
Your voices are no more.

RICHARD:
No more.

ANGELO:
What have you done?

LADY CAPULET:
I am hanged.

ANGELO:
What have you done?

RICHARD:
You know not;
You know not.

ANGELO:
Nay, I do.

LADY CAPULET:
You know not,
You know not, Richmond.

ANGELO:
Your voices are no more.

RICHARD:
No more.

LADY CAPULET:
No more.

ANGELO:
You have done it, you know not;
You have not, you know not, you know not.

LADY CAPULET:
You have not done it, you know not;
There is no more to it.

ANGELO:
No
LORD:
O, the heavens do not think the devil hath set us down
So low,
As we have set down,
So low,
So low,
So low,
To take an upright stand!

CORIOLANUS:
Well, sir, I'll undertake
The exercise of my power in Rome.

SICINIUS:
That's my mind.

CORIOLANUS:
My mind!

SICINIUS:
I'll undertake it with my mind.

CORIOLANUS:
My mind!

SICINIUS:
I'll undertake it with my mind.

CORIOLANUS:


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
print(f'gen_file=${gen_file}')
gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

gen_file=$gpt2_gentext_20211209_030022.txt


In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

##########################################
## Generate Text From The Pretrained Model
## NOT using Fine-Tuned Model at all!
##########################################

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

# Make sure that you 'restart the runtime and then jump here to start

In [ ]:
USE_PRE_TRAINED_MODEL=False
if USE_PRE_TRAINED_MODEL:
  model_name = "774M"
  gpt2.download_gpt2(model_name=model_name)
  sess2 = gpt2.start_tf_sess()
  gpt2.load_gpt2(sess2, model_name=model_name, reuse=False)
  gpt2.generate(sess2,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
# !kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.